In [9]:
import requests
import pandas as pd
from datetime import datetime
import time
from datetime import timedelta
import pickle
import re
import preprocessor as p
from nltk.corpus import stopwords
import pymorphy2
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import warnings
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import collections
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from IPython.display import Markdown as md

# Part 1

### Retrieving posts 
Posts were retrieved from the following vk groups: 
* https://vk.com/itmomagistry 
* https://vk.com/careers_service_itmo 
* https://vk.com/itmoru 
* https://vk.com/itmo_exchange 
* https://m.vk.com/scicomm).

In [217]:
def get_posts(group):

    count = 100
    offset = 0 
    posts = []
    retrieved_posts = 'yes'
    
    while retrieved_posts:

        url = 'https://api.vk.com/method/wall.get'
        params = {
            'domain': group,
            'filter': 'owner',
            'count': count,
            'offset': offset,
            'access_token': '8e84f1f18e84f1f18e84f1f1db8eeb68e388e848e84f1f1d0bb32bd2b9ea45a67630377',
            'v': 5.73
        }

        retrieved_posts = requests.get(url, params = params).json()['response']['items']

        posts += retrieved_posts 
        offset += count   
        time.sleep(0.5)
            
    return posts

In [218]:
groups = ['itmomagistry', 'careers_service_itmo', 'itmoru', 'itmo_exchange', 'scicomm']

posts = []
for group in groups:
    posts += get_posts(group=group)

In [10]:
# pickle.dump( posts, open( "posts.p", "wb" ) )
posts = pickle.load( open( "posts.p", "rb" ) )

### Splitting posts into old (older than a year ago) and new (later than a year ago) ones. 
The old ones will be used for training. The new ones will be used for testing. 

In [11]:
def split_train_test(posts):
    test = []
    train = []
    
    year_ago = (datetime.now()-timedelta(days=365)).timestamp()
    
    for post in posts:
        if post['date'] > year_ago:
            test.append(post)
        else:
            train.append(post)
            
    return test, train

In [12]:
test_posts, train_posts = split_train_test(posts)

In [13]:
md(f'We have retrieved {len(posts)} posts.<br>{len(train_posts)} from them were published before February 2019. These posts will be used for training<br>{len(test_posts)} were published after February 2019. These posts will be used for testing')

We have retrieved 13586 posts.<br>11105 from them were published before February 2019. These posts will be used for training<br>2481 were published after February 2019. These posts will be used for testing

#### Post example

In [14]:
train_posts[0]

{'id': 3764,
 'from_id': -54201931,
 'owner_id': -54201931,
 'date': 1549961199,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '✨День российской науки.\nПрофессиональный праздник отмечается в честь основания в нашей стране Академии наук. Она была создана 295 лет назад.\n📝 Подробности читайте по ссылке: \nhttps://www.1tv.ru/news/2019-02-08/360076-segodnya_den_rossiyskoy_nauki',
 'attachments': [{'type': 'video',
   'video': {'access_key': '27143b75f5296c628d',
    'can_comment': 0,
    'can_like': 1,
    'can_repost': 1,
    'can_subscribe': 1,
    'can_add_to_faves': 1,
    'can_add': 1,
    'comments': 0,
    'date': 1549961199,
    'description': 'Профессиональный праздник отмечается в\xa0честь основания в\xa0нашей стране Академии наук. Она была создана 295 лет назад.',
    'duration': 0,
    'photo_130': 'https://sun9-59.userapi.com/c852216/v852216335/b53c1/ADxFBZJ6jwo.jpg',
    'photo_320': 'https://sun9-11.userapi.com/c852216/v852216335/b53c3/n1zeZST6cbU.jpg',
    'photo_640

# Part 2 

###  Retrieving text from posts and getting rid of duplicate posts

In [15]:
def get_post_text(post):
    if 'copy_history' in post:
        text = post['copy_history'][0]['text']
    else:
        text = post['text']
    return text.lower()

In [16]:
test_texts = {get_post_text(post) for post in test_posts}
train_texts = {get_post_text(post) for post in train_posts}

In [17]:
md(f'Found {len(train_posts) - len(train_texts)} duplicate posts from test set and {len(test_posts) - len(test_texts)} from train set.')

Found 622 duplicate posts from test set and 101 from train set.

### Filtering out posts that mention internships, scholarships, grants etc.

In [18]:
def get_texts_by_keywords(texts, words = {'стажировк', 'стажер', 'стажёр', 'scholarship', 'стипенд', 'грант', 'intern', 'обмен'}):
    return [text for text in texts if any(word in text for word in words)]

In [19]:
train_texts = get_texts_by_keywords(train_texts)

### Dumping 500 of selected posts to excel sheet in order to label them manually.

In [66]:
pd.DataFrame(zip(train_texts[:500], [0]*500)).to_excel('train_dump.xlsx', index = False)

# Part 3

In the following section we'll perform text classification.
<br>**Input**: texts of posts mentionning internships, scholarships etc. with a corresponding class (labeled manually)
<br>**Output**: one of the following classes


#### Classes:
* 0 - other
* 1 - internships
* 3 - scholarships, grants

### Retrieving labeled data from excel sheet.

In [67]:
df = pd.read_excel('train.xlsx')
X_not_processed = df[0]
y = df[1]

### Preprocessing data:
* removing urls, emojis and numbers
* removing punctuation signs
* lemmatization 
* removing stopwords

In [68]:
def preprocess(text, morph, stopwords):
    p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.NUMBER)
    text = p.clean(text)

    words = [morph.parse(word)[0].normal_form for word in re.findall(r'\w+', text)]
    words = [word for word in words if word not in stopwords]
    return " ".join(words)

In [69]:
morph = pymorphy2.MorphAnalyzer()
# russian_stopwords = get_stop_words('ru')
russian_stopwords = stopwords.words("russian")

#### Example
**Before preprocessing:**

In [70]:
X_not_processed[0]

'🔆 фотоника 🔆 оптика 🔆 лазеры🔆\n\nновость - бомба 💣 специально для вас! \n\nпри поддержке организаторами олимпиады «я - профессионал» в пятницу, 14 декабря, организована экскурсия в компанию «лазерные системы» (http://www.lsystems.ru/company/) в пос. стрельна. \n«лазерные системы» – современное инновационное предприятие, работающее в сфере лазерных технологий и оптоэлектронных систем.\nна экскурсии вы не только ближе познакомитесь с компанией и посмотрите производство, но и узнаете о своих карьерных возможностях в её рамках (стажировки, практики, вакансии).\n\nначало экскурсии в 13:00, окончание в 17:00. развозка осуществляется от м. проспект ветеранов на заказном автобусе. \n\n⚠поторопитесь! заявки принимаются до 10:00 завтрашнего утра. \n \n📌количество мест сильно ограничено. поэтому скорее зарегистрируйтесь https://goo.gl/forms/pmd7zlmdl62fp0gn1 \n \nпосле регистрации с вами свяжутся для обсуждения деталей экскурсии и места встречи.\n\nкоординатор встречи - рябцева юстина, career.ip

**After preprocessing:**

In [72]:
preprocess(X_not_processed[0], morph, russian_stopwords)

'фотоник оптик лазер новость бомба специально поддержка организатор олимпиада профессионал пятница декабрь организовать экскурсия компания лазерный система пос стрельна лазерный система современный инновационный предприятие работать сфера лазерный технология оптоэлектронный система экскурсия близкий познакомиться компания посмотреть производство узнать свой карьерный возможность её рамка стажировка практика вакансия начать экскурсия 00 окончание 00 развозка осуществляться метр проспект ветеран заказной автобус поторопиться заявка приниматься 00 завтрашний утро количество место сильно ограничить поэтому скорее зарегистрироваться регистрация связаться обсуждение деталь экскурсия место встреча координатор встреча рябцева юстина career iprofi yandex ru 7 495 916 89 00 доб 7 985 100 89 84'

In [73]:
X = [preprocess(text, morph, russian_stopwords) for text in X_not_processed]

### Splitting train data into training and validation sets.

In [74]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

### Choosing the best classifier

In [75]:
vectorizers = {'TfidfVectorizer':TfidfVectorizer(), 
               'CountVectorizer':CountVectorizer()
              }
classifiers = {'GradientBoostingClassifier':GradientBoostingClassifier(random_state = 0), 
               'RandomForestClassifier':RandomForestClassifier(random_state = 0), 
               'LinearSVC':LinearSVC(random_state = 0), 
               'MLPClassifier':MLPClassifier(random_state = 0)}

In [76]:
f1_scores = {}
for vectorizer_name, vectorizer in vectorizers.items():
    scores = {}
    X_train_vec = vectorizer.fit_transform(X_train)
    X_val_vec = vectorizer.transform(X_val)
    for classifier_name, classifier in classifiers.items():
        classifier.fit(X_train_vec, y_train)
        predictions = classifier.predict(X_val_vec)
        scores[classifier_name] = f1_score(y_val, predictions, average = 'weighted')
    f1_scores[vectorizer_name] = scores

### Classification results

In [77]:
pd.DataFrame(f1_scores)

,TfidfVectorizer,CountVectorizer
GradientBoostingClassifier,0.783664,0.778332
LinearSVC,0.810141,0.774514
MLPClassifier,0.773646,0.780671
RandomForestClassifier,0.738774,0.753181


Looks like the most promissing results are achieved by the **GradientBoostingClassifier** together with **TfidfVectorizer**. 
<br>LinearSVC together with **TfidfVectorizer** gives a slightly worse result while being much simpler than GradientBoostingClassifier. The latter looks too sophisticated for our simple problem, so let's stick to the LinearSVC model and try to improve this model's performance by tuning hyperparameters using GridSearchCV.

In [78]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

In [79]:
param_grid = {'C':np.arange(0.01,100,10), 'max_iter':[1, 5, 10, 50, 100]}
gs = GridSearchCV(LinearSVC(),param_grid,cv=5,return_train_score=True)
gs.fit(X_train_vec,y_train)
best_parameters = gs.best_params_

In [80]:
print(f'Best parameters: {best_parameters}')

Best parameters: {'C': 80.01, 'max_iter': 5}


In [81]:
clf = LinearSVC(**best_parameters, random_state = 0)
clf.fit(X_train_vec,y_train)
predictions = clf.predict(X_val_vec)

In [82]:
best_f1_score = np.around(f1_score(y_val, predictions, average = 'weighted'), decimals=2)

In [83]:
print(f'F1 score of the model with tuned hyperparameters: {best_f1_score}')

F1 score of the model with tuned hyperparameters: 0.8


# Part 4
In this section we'll try to detect topics of posts about internships and about scholarships.
<br>First, let's try to extract topics automatically using LDA model.

In [84]:
def lda(values, num_topics=3, num_words=5):
    values = [preprocess(text, morph, russian_stopwords).split() for text in values]
    dictionary = corpora.Dictionary(values)
    corpus = [dictionary.doc2bow(text) for text in values]
    ldamodel = LdaModel(corpus, num_topics, id2word=dictionary, passes=15)
    topics = ldamodel.print_topics(num_words)
    return topics

In [85]:
selected_posts = {'internships': [post for post, c in zip(X_not_processed, predictions) if c == 1],
                  'scholarships': [post for post, c in zip(X_not_processed, predictions) if c == 3]}

In [86]:
for type, values in selected_posts.items():
    print(type, '\n')
    topics = lda(values)
    for i, topic in enumerate(topics):
        print(i, topic)
    print()

internships 

0 (0, '0.011*"стажировка" + 0.011*"компания" + 0.008*"работа" + 0.007*"практика" + 0.007*"итмый" + 0.007*"лазерный" + 0.006*"система" + 0.006*"это" + 0.005*"разработка" + 0.005*"опыт"')
1 (1, '0.016*"университет" + 0.016*"итмый" + 0.008*"студент" + 0.007*"год" + 0.007*"проект" + 0.007*"который" + 0.006*"заявка" + 0.006*"получить" + 0.006*"стажировка" + 0.006*"программа"')
2 (2, '0.010*"итмый" + 0.009*"стажировка" + 0.009*"компания" + 0.008*"университет" + 0.007*"свой" + 0.006*"студент" + 0.006*"который" + 0.006*"работа" + 0.005*"международный" + 0.005*"научный"')

scholarships 

0 (0, '0.015*"итмый" + 0.010*"работа" + 0.008*"стажировка" + 0.008*"itmo" + 0.008*"мероприятие" + 0.008*"крупный" + 0.008*"компания" + 0.008*"участие" + 0.008*"superjob" + 0.008*"университет"')
1 (1, '0.017*"итмый" + 0.015*"университет" + 0.009*"студент" + 0.009*"itmo" + 0.009*"год" + 0.009*"стажировка" + 0.009*"компания" + 0.008*"стипендия" + 0.008*"научный" + 0.008*"дверь"')
2 (2, '0.020*"компан

Unfortunately, there doesn't seem to be any topics that could be easily extracted by the model (even if we tune number of topics). So let's use the good old string matching to find **the most wanted skills** in the intership posts.

First, let's look at the most popular words in the posts.

In [87]:
internship_texts = [preprocess(text, morph, russian_stopwords) for text in selected_posts['internships']]
internship_words = [text.split() for text in internship_texts]
internship_words = [item for sublist in internship_words for item in sublist]
counter = collections.Counter(internship_words)
print(counter.most_common(200))

[('итмый', 72), ('университет', 65), ('стажировка', 49), ('компания', 47), ('студент', 39), ('свой', 33), ('работа', 33), ('проект', 32), ('который', 32), ('это', 28), ('получить', 25), ('международный', 25), ('год', 25), ('itmo', 25), ('программа', 24), ('возможность', 22), ('заявка', 21), ('весь', 21), ('конкурс', 20), ('участие', 19), ('опыт', 18), ('система', 18), ('практика', 16), ('грант', 16), ('команда', 15), ('россия', 15), ('узнать', 15), ('центр', 15), ('область', 15), ('открытый', 15), ('научный', 15), ('регистрация', 15), ('разработка', 14), ('пройти', 14), ('ru', 14), ('мочь', 13), ('самый', 12), ('смочь', 12), ('крупный', 12), ('лазерный', 12), ('вопрос', 12), ('лаборатория', 12), ('день', 12), ('курс', 12), ('дробный', 12), ('обучение', 12), ('ждать', 12), ('вуз', 12), ('подробность', 11), ('специалист', 11), ('вакансия', 11), ('российский', 11), ('технология', 11), ('институт', 11), ('сотрудник', 11), ('конференция', 11), ('spie', 11), ('стипендия', 11), ('социальный',

### Now let's choose the words that correspond to some programmer's skills and are relatively popular in the dataset.

In [88]:
key_words = ['java', 'c', 'python', 'javascript', 'frontend', 'backend', 'linux', 'office',
             'bi', 'data science', 'анализ дать', 'машинный обучение', 'machine learning', 'аналитик', 
             'тестировщик', 'тестирование', 'английский', 'дизайн', 'разработка', 'разработчик', 'инженер','1с']

In [89]:
def get_most_popular_skills(key_words, texts):
    skills_dict = dict.fromkeys(key_words, 0)
    for text in texts:
        for word in key_words:
            if word in text:
                skills_dict[word] += 1
    skills_dict = {skill: count for skill, count in sorted(skills_dict.items(), key=lambda item: item[1], reverse=True)}
    return skills_dict

In [90]:
skills_dict = get_most_popular_skills(key_words, internship_texts)
for skill, count in skills_dict.items():
    print(f'{skill}: {count}')

c: 39
разработка: 10
дизайн: 6
инженер: 6
bi: 4
тестирование: 4
английский: 4
разработчик: 3
python: 1
frontend: 1
анализ дать: 1
машинный обучение: 1
machine learning: 1
аналитик: 1
java: 0
javascript: 0
backend: 0
linux: 0
office: 0
data science: 0
тестировщик: 0
1с: 0


# Part 5
## Test data
Now let's take a look at the test set, i.e. texts published before Ferbuary 2019.

#### First, let's use our classifier to extract texts that announce internships and scholarships. 

In [91]:
test_texts_orig = get_texts_by_keywords(test_texts)
test_texts = [preprocess(text, morph, russian_stopwords) for text in test_texts_orig]

In [92]:
test_texts_vec = vectorizer.transform(test_texts)
test_predictions = clf.predict(test_texts_vec)

#### Now let's dump texts and our predictions to a csv file.

In [93]:
pd.DataFrame(zip(test_texts_orig, test_predictions), 
             columns = ['text', 'class']).to_csv('part_3.csv', index=False)

In [94]:
test_internships = [post for post, c in zip(test_texts, test_predictions) if c == 1]

#### What are the most wanted skills in 2019-2020?

In [95]:
test_skills_dict = get_most_popular_skills(key_words, test_internships)
for skill, count in test_skills_dict.items():
    print(f'{skill}: {count}')

c: 68
разработчик: 18
разработка: 17
инженер: 17
аналитик: 14
английский: 13
bi: 12
тестирование: 11
java: 10
python: 9
frontend: 6
backend: 4
дизайн: 4
javascript: 3
linux: 3
data science: 3
машинный обучение: 3
анализ дать: 1
тестировщик: 1
office: 0
machine learning: 0
1с: 0


### What can we tell from the data?
* Most of the companies were looking for developers, most of them for c and c++ languages.
* Machine Learning, python, data science, business intelligence speciallists are much more likely to find an internship in 2020 than in the previous years.

In [96]:
def get_keywords_from_text(key_words, text):
    result = []
    for word in key_words:
        if word in text:
            result.append(word)
    return result

In [97]:
test_internships_keywords = [get_keywords_from_text(key_words, text) for text in test_internships]

#### Now let's dump internships texts and our predictions to a csv file.

In [98]:
test_internships_orig = [post for post, c in zip(test_texts_orig, test_predictions) if c == 1]
pd.DataFrame(zip(test_internships_orig, test_internships_keywords), 
             columns = ['text', 'keywords']).to_csv('part_4.csv', index=False)